In [1]:
# '.'             -- prazno polje
# 'Neki broj'     -- blago
# 'x'             -- klopka
mapa = [['.','x','.','.','-32'],
    ['3','50','.','.','.'],
    ['.','200','.','-55','.'],
    ['.','x','.','.','.'],
    ['.','x','20','.','45']]
velicina_mape = 5

In [2]:
import copy

class Chromosome:
    '''
    Predstavlja jedan hromozom, odnosno jednu nisku.
    Niska je data genetskim kodom i ocenom prilagodjenosti 
    (u ovom primeru, broj karaktera date niske koji se poklapaju sa ciljnom niskom)
    '''
    def __init__(self, genetic_code, fitness):
        self.genetic_code = genetic_code
        self.fitness = fitness
    
    def __str__(self):
        return "{} = {}".format(self.genetic_code, self.fitness)
    
    def draw_board(self):
        pomocna_mapa = copy.deepcopy(mapa)
        pozicijaVauVau_i = 0
        pozicijaVauVau_j = 0
        blago = 0
        
        for i in range(100):
            index_i = self.genetic_code[i][0]
            index_j = self.genetic_code[i][1]
            
            # provera da li je polje pored
            if abs(pozicijaVauVau_i - index_i) + abs(pozicijaVauVau_j - index_j) > 1:
                pomocna_mapa[pozicijaVauVau_i][pozicijaVauVau_j] = 'W'
                break
            
            # ako je polje poseceno, izlazi se
            if pomocna_mapa[index_i][index_j] == 'o':
                pomocna_mapa[pozicijaVauVau_i][pozicijaVauVau_j] = 'W'
                break
            
            # klopka
            if pomocna_mapa[index_i][index_j] == 'x':
                print('Stao na klopku!!')
                pomocna_mapa[index_i][index_j] = 'W'
                pomocna_mapa[pozicijaVauVau_i][pozicijaVauVau_j] = 'o'
                break
            
            # stigao na cilj
            if index_i == velicina_mape-1 and index_j == velicina_mape-1:
                pomocna_mapa[pozicijaVauVau_i][pozicijaVauVau_j] = 'o'
                pomocna_mapa[index_i][index_j] = 'W'
                break
            
            # nista od navedenog
            if pomocna_mapa[index_i][index_j] != '.':
                blago += int(pomocna_mapa[index_i][index_j])
                
            pomocna_mapa[pozicijaVauVau_i][pozicijaVauVau_j] = 'o'
            pozicijaVauVau_i = index_i
            pozicijaVauVau_j = index_j
            
        for i in range(velicina_mape):
            for j in range(velicina_mape):
                print("{:<5}|".format(pomocna_mapa[i][j]), end="")
            print()

In [3]:
import random
import string

class GeneticAlgorithm:
    def __init__(self):                       
        self.chromosome_size = velicina_mape*velicina_mape            # Duzina niske koja se pogadja
        
        '''Parametri genetskog algoritma, eksperimentalno izabrani.'''
        self.generation_size = 1000         # Broj jedinki u jednoj generaciji
        self.reproduction_size = 500        # Broj jedinki koji ucestvuje u reprodukciji    
        self.max_iterations = 1000          # Maksimalni dozvoljeni broj iteracija
        self.mutation_rate = 0.0001         # Verovatnoca da se desi mutacija
        self.tournament_size = 30           # Velicina turnira
        self.cross_rate = 0.6               # Verovatnoca za odabir bita prvog roditelja pri ukrstanju
        
        self.possible_gene_values = velicina_mape   # Velicina mape odredjuje indekse koji se mogu odabrati

    '''Vraca blago+predjeni_put koje je VauVau sakupio. Ako je stigao do cilja vraca blago+predjeni_put+100, 
       a ako je stao na klopku vraca -100. '''
    def calculate_fitness(self, hromozom):
        fitness_value = 0
        pozicijaVauVau_i = 0
        pozicijaVauVau_j = 0
        
        pomocna_mapa = copy.deepcopy(mapa)
        
        for i in range(self.chromosome_size):
            index_i = hromozom[i][0]
            index_j = hromozom[i][1]
            
            # provera da li je polje pored
            if abs(pozicijaVauVau_i - index_i) + abs(pozicijaVauVau_j - index_j) > 1:
                return fitness_value
            
            # ako je polje poseceno, izlazi se
            if pomocna_mapa[index_i][index_j] == 'o':
                return fitness_value
            
            # klopka
            if pomocna_mapa[index_i][index_j] == 'x':
                return -100
            
            # stigao na cilj
            if index_i == velicina_mape-1 and index_j == velicina_mape-1:
                return fitness_value + 100
            
            # nista od navedenog
            if pomocna_mapa[index_i][index_j] != '.':
                fitness_value += int(pomocna_mapa[index_i][index_j])
            else:
                fitness_value += 1
                
            pomocna_mapa[pozicijaVauVau_i][pozicijaVauVau_j] = 'o'
            pozicijaVauVau_i = index_i
            pozicijaVauVau_j = index_j
        
        return fitness_value
        
    '''Generise se _generation_size nasumicnih jedinki.'''
    def initial_population(self):
        init_population = []
        
        for i in range(self.generation_size):
            # generise se i-ti nasumicni genetski kod
            hromozom = []
            poz_i = 0
            poz_j = 0
            hromozom.append([poz_i, poz_j])
            
            for j in range(self.chromosome_size - 1):               
                smer = [[0, 1], [0, -1], [1, 0], [-1, 0]]
                lista_izbora = []
                for k in range(4):
                    if (poz_i + smer[k][0] >= 0 and poz_i + smer[k][0] < velicina_mape and
                        poz_j + smer[k][1] >= 0 and poz_j + smer[k][1] < velicina_mape):
                        lista_izbora.append(smer[k])
                    
                smer_slucajan = random.choice(lista_izbora)
                poz_i = poz_i + smer_slucajan[0]
                poz_j = poz_j + smer_slucajan[1]
                hromozom.append([poz_i, poz_j])
            
            # izracunava se prilagodjenost generisane niske
            fitness = self.calculate_fitness(hromozom)
            new_chromosome = Chromosome(hromozom, fitness)
            
            # dodaje se u populaciju
            init_population.append(new_chromosome)
            
        return init_population
    
    '''Funkcija za izbor hromozoma za reporodukciju'''
    def selection(self, chromosomes):
        selected = []
        
        # Bira se self.reproduction_size hromozoma za reprodukciju
        # Selekcija moze biti ruletska ili turnirska
        for i in range(self.reproduction_size):
            selected.append(self.tournament_selection(chromosomes))
          
        # Vracaju se izabrani hromozomi za repodukciju
        return selected
    
    
    '''Bira jednu jedinku koristeci turnirsku selekciju. '''
    def tournament_selection(self, chromosomes):
        
        #------------------------------------------------------------------------------------------
        # STUDENTSKI KOD
        #------------------------------------------------------------------------------------------

    
    '''Vrsi mutaciju nad hromozomom sa verovatnocom self._mutation_rate.
       Mutacija se vrsi nad jednim genom (karakterom) sa proizvoljnim indeksom.
    '''
    def mutate(self, hromozom):
        random_value = random.random()
        
        # ukoliko je ispunjen uslov, izvrsi mutaciju
        if random_value < self.mutation_rate:
            
            # izabrati proizvoljan indeks
            random_index = random.randrange(self.chromosome_size)
            
            while True:
                index = random.randrange(2)
                new_value = random.randrange(self.possible_gene_values)
                
                if hromozom[random_index][index] != new_value:
                    break
                    
            hromozom[random_index][index] = new_value
            
        return hromozom
        
    '''Od jedinki generise novu generaciju primenjujuci genetske operatore 
       ukrstanje (crossover) i mutaciju (mutation). 
    '''
    def create_generation(self, chromosomes):
        #------------------------------------------------------------------------------------------
        # STUDENTSKI KOD
        #------------------------------------------------------------------------------------------
            
    ''' Vrsi uniformno ukrstanje po verovatnoci self._crossover_p i vraca 2 nove jedinke.'''
    def crossover(self, parent1, parent2):
        #------------------------------------------------------------------------------------------
        # STUDENTSKI KOD
        #------------------------------------------------------------------------------------------
        
    '''Izvrsavanje genetskog algoritma'''
    def optimize(self):
        #------------------------------------------------------------------------------------------
        # STUDENTSKI KOD
        #------------------------------------------------------------------------------------------
            
        global_best_chromosome.draw_board()
        return global_best_chromosome

In [4]:
g = GeneticAlgorithm()

In [5]:
g.optimize()

o    |x    |.    |.    |-32  |
o    |o    |.    |.    |.    |
.    |o    |o    |-55  |.    |
.    |x    |W    |.    |.    |
.    |x    |20   |.    |45   |
